In [1]:
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os

In [2]:
root_path  = "//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/DICOM_data/DATA_VOLLEDIG_unstructured/"
PxList = np.sort(os.listdir(root_path))

In [21]:
def FindCT_UID(path):
    files = os.listdir(path)
    ct_dicom_dataset = pydicom.dcmread(os.path.join(path,files[0]))
    series_instance_uid = ct_dicom_dataset[(0x0020, 0x0052)].value
    return series_instance_uid

In [22]:
def FindRT_UID_perContour(path,roi_name_to_find):
    files = os.listdir(path)
    rtstruct_dataset = pydicom.dcmread(path+files[0])
    if rtstruct_dataset.Modality == "RTSTRUCT":
        roi_sequence = rtstruct_dataset.StructureSetROISequence
        for roi_item in roi_sequence:
            roi_number = roi_item.ROINumber
            roi_name = roi_item.ROIName
            if roi_name == roi_name_to_find:
                # Print information for the found ROI
                #print(f"ROI Name: {roi_name}")
                #print(f"ROI Number: {roi_number}")
                referenced_frame_of_reference_uid = roi_item.ReferencedFrameOfReferenceUID
                #print(f"Referenced Frame of Reference UID: {referenced_frame_of_reference_uid}")
                return referenced_frame_of_reference_uid

        else:
            return None


In [23]:
def GetTheContour(ctfolder,rtfolder,label):
    rtfiles = os.listdir(rtfolder)
    rtstruct = RTStructBuilder.create_from(
        dicom_series_path=ctfolder,
        rt_struct_path=rtfolder+rtfiles[0]
    )
    mask_3d = rtstruct.get_roi_mask_by_name(label)

In [27]:
def FindUIDMatch(RtPaths,CTPaths,labels):
    numcontours=0
    for currRTpath in RTStruct_fpaths:
        for currCTpath in CTPaths:
            for label in labels:
                ctUID = FindCT_UID(currCTpath)
                rtUID = FindRT_UID_perContour(currRTpath,label)
                if rtUID == ctUID and ctUID is not None and rtUID is not None:
                    numcontours+=1
                    #print("Match",currCTpath)
    return numcontours

In [30]:
for Px in PxList:
    AvgCt_fpaths = []
    RTStruct_fpaths = []
    BP50_CT_fpaths,BP60_CT_fpaths= ([],[])
    BP10_CT_fpaths,BP90_CT_fpaths= ([],[])
    num50GTVs,numITVs,num60GTVs = (0,0,0)
    #Look for the Paths
    for root, dirs, files in os.walk(root_path+Px, topdown=True):
        for d in dirs:
            if "Thorax" in d or "Ave" in d:
                AvgCt_fpaths.append(root+'/'+d+'/')
            if ('RTSTRUCT' in d or "pproved" in d) or ("PinnPlan" in d and not("RTDOSE" in d)):
                RTStruct_fpaths.append(root+'/'+d+'/')
            if "50%" in d and not "MIP" in d and not "Ave" in d and not "Thorax" in d: BP50_CT_fpaths.append(root+'/'+d+'/')
            if "60%" in d and not "MIP" in d and not "Ave" in d and not "Thorax" in d: BP60_CT_fpaths.append(root+'/'+d+'/')
            if "10%" in d and not "MIP" in d and not "Ave" in d and not "Thorax" in d: BP10_CT_fpaths.append(root+'/'+d+'/')
            if "90%" in d and not "MIP" in d and not "Ave" in d and not "Thorax" in d: BP90_CT_fpaths.append(root+'/'+d+'/')
    
    #GET the UIDs
    GTVlabels = ['GTV']
    ITVlabels = ['ITV']
    numITVs = FindUIDMatch(RTStruct_fpaths,AvgCt_fpaths,ITVlabels)
    num50GTVs = FindUIDMatch(RTStruct_fpaths,BP50_CT_fpaths,GTVlabels)
    num60GTVs = FindUIDMatch(RTStruct_fpaths,BP60_CT_fpaths,GTVlabels)
    print("Px:",Px,"ITVs Matched:",numITVs,"GTV 50%:",num50GTVs,"GTV 60%:",num60GTVs)
    if num50GTVs != 0 or num60GTVs != 0:
        break

Px: 0004879 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Px: 0010539 ITVs Matched: 1 GTV 50%: 0 GTV 60%: 1
